In [2]:
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np

In [17]:
data_df = pd.read_csv("./PreferenciasBritanicos.csv", header=0)

class NaiveBayesClassifier:

    def __init__(self):
        self.relative_frequencies = {} #by_class_by_value
        self.classes_probabilities = {}

    def train(self, data_df, classes, classes_column_name):
        for c in classes:
            class_df = data_df[data_df[classes_column_name] == c].drop(classes_column_name, axis=1)
            self.classes_probabilities[c] = len(class_df) / len(data_df)

            values_appearances = {column_name: {} for column_name in class_df.columns}
            # iterate over rows within class
            for i in range(len(class_df)):
                row = class_df.iloc[[i]]

                # iterate over values within row
                for column_name in row.columns:
                    # initialize possible value if not present in map, otherwise increment appereances
                    column_value = row[column_name].values[0]

                    if column_value not in values_appearances[column_name]:
                        values_appearances[column_name][column_value] = 1
                    else:
                        values_appearances[column_name][column_value] += 1

            # calculate relative frequencies
            self.relative_frequencies[c] = { column_name: {
                key: (value + 1) / (len(class_df) + len(classes)) for key, value in values_appearances[column_name].items()
            } for column_name in class_df.columns}


    def classify(self, sample):
        maximizing_class = None
        maximizing_prod = -1
        # for c in classes:
        #     prod = self.classes_probabilities[c] * self.relative_frequencies_by_class[c].prod()

        #     if(prod > maximizing_prod):
        #         maximizing_prod = prod
        #         maximizing_class = c
        

    

NaiveBayesClassifier().train(data_df, ["I", "E"], "Nacionalidad")




dict_items([(0, 3), (1, 3)])
dict_items([(1, 7)])
{'scones': {0: 0.5, 1: 0.5}, 'cerveza': {0: 0.5, 1: 0.5}, 'wiskey': {1: 0.375, 0: 0.625}, 'avena': {1: 0.5, 0: 0.5}, 'futbol': {1: 0.5, 0: 0.5}}
{'scones': {1: 0.8888888888888888}, 'cerveza': {0: 0.4444444444444444, 1: 0.5555555555555556}, 'wiskey': {0: 0.5555555555555556, 1: 0.4444444444444444}, 'avena': {1: 0.6666666666666666, 0: 0.3333333333333333}, 'futbol': {1: 0.4444444444444444, 0: 0.5555555555555556}}
{'I': 0.46153846153846156, 'E': 0.5384615384615384}
1.0
0.9401709401709402
